In [1]:
import sys
import pandas as pd
import numpy as np
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
from ckiptagger import construct_dictionary
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

host = 4
host = 0

if host == 0:
    path = r'/Users/aron/Documents/GitHub/Perfume/2_NLP'
    dict_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/dict.xlsx'
    data_file = r'/Users/aron/Documents/GitHub/Perfume/1_Crawler/Resource/data.xls'
    
elif host == 4:
    path = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/'
    dict_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/dict.xlsx'
    data_file = r'/home/rserver/Data_Mining/personal_workspace/yz/Lab/CkipTagger/Resource/data.xls'

    
# Codebase ......
path_codebase = [r'/Users/aron/Documents/GitHub/Arsenal/',
                 r'/Users/aron/Documents/GitHub/Codebase_YZ/',
                 r'/home/aronhack/stock_predict/Function',
                 r'D:\GitHub\Arsenal',
                 r'D:\Data_Mining\Projects\Codebase_YZ',
                 r'/home/jupyter/Arsenal/20220522',
                 path + '/Function']


for i in path_codebase:    
    if i not in sys.path:
        sys.path = [i] + sys.path
    

import codebase_yz as cbyz
    
    
path_resource = path + '/Resource'
path_export = path + '/Export'

## Import Custom Dictionary

In [2]:
dict_df = pd.read_excel(dict_file)
dict_df = cbyz.df_lower(df=dict_df, cols=[])
dict_li = dict_df['word'].tolist()
custom_dict = dict((el, 1) for el in dict_li)
custom_dict = construct_dictionary(custom_dict)

In [3]:
# 2. Load model
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
# https://drive.google.com/drive/folders/105IKCb88evUyLKlLondvDBoh7Dy_I1tm

ws = WS(path_resource + "/data")
pos = POS(path_resource + "/data")
ner = NER(path_resource + "/data")

/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/Users/aron/opt/anaconda3/lib/python3.8/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMC

## 4. Run the WS-POS-NER pipeline

In [199]:
data = pd.read_excel(data_file)

# Preprocessing
data = data.dropna(subset=['title'], axis=0)
data = data[~data['title'].str.contains('廣告')]
data = cbyz.df_lower(df=data, cols=['title', 'brand'])
data['title'] = data['title'].apply(cbyz.unicode_filter)

# There are bug in the crawler, so clean title by limiting length of string.
data = data[(data['title'].str.len()>=5) & (data['title'].str.len()<100)]

# data = data[~data['name'].isna()]
# data = data[data.index<=300]

data = data.dropna(subset=['title'], axis=0)
data['title'] = data['title'].str.replace('找相似', '')

print('Bug - title裡面還是有括號')

Bug - title裡面還是有括號


In [200]:
y = 'name'
y = 'brand'
y = 'type'

In [265]:
train_raw = data[~data[y].isna()]
title_train = train_raw['title'].tolist()

pred_raw = data[data[y].isna()]
title_pred = pred_raw['title'].tolist()

y_train = train_raw[y].tolist()

In [266]:
word_sentence_train = ws(
    title_train,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_train = pos(word_sentence_train)
entity_sentence_train = ner(word_sentence_train, pos_sentence_train)

In [135]:
word_sentence_pred = ws(
    title_pred,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = custom_dict, # words in this dictionary are forced
)

pos_sentence_pred = pos(word_sentence_pred)
entity_sentence_pred = ner(word_sentence_pred, pos_sentence_pred)

In [205]:
# df = pd.DataFrame()
# for i, sentence in enumerate(y_train):
    
#     cur_sentence = word_sentence_train[i]
#     cur_y = y_train[i]
    

#     # cur_y不是na
#     cur_tag = []
#     for j in cur_sentence:

#         if y == 'type':
#             if j == cur_y:
#                 cur_tag.append(1)
#             else:
#                 cur_tag.append(0)
#         else:
#             if j in cur_y:
#                 cur_tag.append(1)
#             else:
#                 cur_tag.append(0)
                
#     new_li = list(zip(cur_sentence,  pos_sentence_list[i], cur_tag))
#     temp_df = pd.DataFrame(new_li, columns=['word', 'pos', 'tag'])
#     temp_df['sentence'] = i
    
#     df = df.append(temp_df)
    
# df = df[['sentence', 'word', 'pos', 'tag']]
# df = cbyz.df_conv_col_type(df=df, cols='tag', to='str')
# df

,sentence,word,pos,tag
0,0,95,D,0
1,0,折,VB,0
2,0,現貨,FW,0
3,0,clausportomusgorealblackedition,Neu,0
4,0,100,Nf,0
...,...,...,...,...
12,35,曠野,D,0
13,35,之,VC,0
14,35,心,Neu,0
15,35,3,Nc,0


In [280]:
def word2df(word_li, y_li=[]):
    
    result = pd.DataFrame()
    for i in range(len(y_li)):

        cur_sentence = word_li[i]
        cur_y = y_train[i]


        # cur_y不是na
        cur_tag = []
        for j in cur_sentence:

            if y == 'type':
                if j == cur_y:
                    cur_tag.append(1)
                else:
                    cur_tag.append(0)
            else:
                if j in cur_y:
                    cur_tag.append(1)
                else:
                    cur_tag.append(0)

        new_li = list(zip(cur_sentence,  pos_sentence_list[i], cur_tag))
        new_result = pd.DataFrame(new_li, columns=['word', 'pos', 'tag'])
        new_result['sentence'] = i

        result = result.append(new_result)

    return result

In [281]:
df_train = word2df(word_li=word_sentence_train, y_li=y_train)
df_train = df_train[['sentence', 'word', 'pos', 'tag']]
df_train = cbyz.df_conv_col_type(df=df_train, cols='tag', to='str')
df_train

# Bug
# df_pred = word2df(word_li=word_sentence_pred, y_li=y_train)
# df_pred = df_pred[['sentence', 'word', 'pos', 'tag']]
# df_pred = cbyz.df_conv_col_type(df=df_pred, cols='tag', to='str')
# df_pred

,sentence,word,pos,tag
0,0,95,D,0
1,0,折,VB,0
2,0,現貨,FW,0
3,0,clausportomusgorealblackedition,Neu,0
4,0,100,Nf,0
...,...,...,...,...
12,35,曠野,D,0
13,35,之,VC,0
14,35,心,Neu,0
15,35,3,Nc,0


In [282]:
len(word_sentence_train), len(y_train)

(36, 36)

In [206]:
word_sentence_train

[['95',
  '折',
  '現貨',
  'clausportomusgorealblackedition',
  '100',
  'ml',
  '淡香水',
  '(',
  '靜謐',
  '黑',
  '典藏版',
  ')',
  '39803781'],
 ['6',
  '折',
  'bvlgari',
  '寶格麗',
  '女性',
  '淡香水',
  '淡香精',
  '(',
  '5',
  'ml',
  ')',
  '紫',
  '水',
  '晶晶',
  '澈晶',
  '艷',
  '玫瑰',
  '金漾',
  '歡沁',
  '玫香',
  '航空版',
  '美麗',
  '購',
  '299359',
  '已',
  '售出',
  '27',
  '苗栗縣',
  '後龍鎮'],
 ['bvlgari',
  '寶格麗',
  '勁',
  '藍',
  '水',
  '能量',
  '男性',
  '淡香水',
  '50',
  'ml',
  '100',
  'ml',
  '娜娜',
  'outlet',
  '情人節',
  '禮物',
  '送',
  '禮',
  '禮物',
  '男香',
  '12991890',
  '已',
  '售出',
  '2',
  '臺中市',
  '大里區'],
 ['現貨',
  '公司貨',
  'bvlgari',
  '寶格麗',
  '淡香水',
  '淡香精',
  '5',
  'ml',
  '歡沁',
  '玫香',
  '玫瑰',
  '金漾白',
  '水',
  '晶',
  '森林',
  '之',
  '光',
  '活力',
  '海洋',
  '能量',
  '299350',
  '已',
  '售出',
  '31',
  '臺中市',
  '大里區'],
 ['4',
  '折',
  'bvlgari',
  '寶格麗',
  '紫',
  '水',
  '晶',
  '花舞',
  '輕盈',
  '女性',
  '淡香水',
  'tester',
  '65',
  'ml',
  '瓶',
  '有',
  '蓋',
  '環保',
  '包裝',
  '現貨',
  '廠商',
  '直',
  '

In [207]:
# 6. Show Results
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return

In [245]:
# for i, sentence in enumerate(sentence_list):
#     print(f"'{sentence}'")
#     print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
#     for entity in sorted(entity_sentence_list[i]):
#         print(entity)

## Feature Extraction

In [209]:
# Worklist
# 1. Add types of perfume in the title

In [254]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['word'].values.tolist(), 
                                                           s['pos'].values.tolist(), 
                                                           s['tag'].values.tolist())]
        self.grouped = self.data.groupby('sentence').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
        
getter_train = SentenceGetter(df_train)
sentence_train = getter_train.sentences

getter_pred = SentenceGetter(df_pred)
sentence_pred = getter_pred.sentences

In [211]:
sentences

[[('95', 'D', '0'),
  ('折', 'VB', '0'),
  ('現貨', 'FW', '0'),
  ('clausportomusgorealblackedition', 'Neu', '0'),
  ('100', 'Nf', '0'),
  ('ml', 'Neu', '0'),
  ('淡香水', 'VCL', '1'),
  ('(', 'Na', '0'),
  ('靜謐', 'Na', '0'),
  ('黑', 'Neu', '0')],
 [('6', 'VJ', '0'),
  ('折', 'VH', '0'),
  ('bvlgari', 'FW', '0'),
  ('寶格麗', 'Na', '0'),
  ('女性', 'Neu', '0'),
  ('淡香水', 'Nf', '0'),
  ('淡香精', 'Neu', '1'),
  ('(', 'VA', '0'),
  ('5', 'Neu', '0')],
 [('bvlgari', 'FW', '0'),
  ('寶格麗', 'Nb', '0'),
  ('勁', 'Na', '0'),
  ('藍', 'Na', '0'),
  ('水', 'Neu', '0'),
  ('能量', 'Nf', '0'),
  ('男性', 'Na', '0'),
  ('淡香水', 'Na', '1'),
  ('50', 'Na', '0'),
  ('ml', 'Na', '0'),
  ('100', 'D', '0'),
  ('ml', 'VD', '0'),
  ('娜娜', 'Neu', '0'),
  ('outlet', 'D', '0'),
  ('情人節', 'VC', '0'),
  ('禮物', 'Neu', '0'),
  ('送', 'Nc', '0'),
  ('禮', 'Nc', '0')],
 [('現貨', 'FW', '0'),
  ('公司貨', 'Nb', '0'),
  ('bvlgari', 'VH', '0'),
  ('寶格麗', 'Na', '0'),
  ('淡香水', 'Na', '0'),
  ('淡香精', 'FW', '0'),
  ('5', 'Nb', '0'),
  ('ml', 'Neu', '0

In [212]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
        
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [235]:
### Split train and test sets
X = [sent2features(s) for s in sentences]
new_y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=0)

# X_pred

### Train a CRF model

In [236]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [237]:
trainer.set_params({
#     'c1': 1.0,   # coefficient for L1 penalty
#     'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

trainer.train(path_export + '/ckip_tagger_model.crfsuite')

In [238]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [239]:
trainer.logparser.last_iteration

{'num': 27,
 'scores': {},
 'loss': 28.272978,
 'feature_norm': 3.693636,
 'error_norm': 0.000895,
 'active_features': 972,
 'linesearch_trials': 2,
 'linesearch_step': 0.171797,
 'time': 0.001}

In [240]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

27 {'num': 27, 'scores': {}, 'loss': 28.272978, 'feature_norm': 3.693636, 'error_norm': 0.000895, 'active_features': 972, 'linesearch_trials': 2, 'linesearch_step': 0.171797, 'time': 0.001}


## Make predictions

To use the trained model, create pycrfsuite.Tagger, open the model and use "tag" method:

In [241]:
tagger = pycrfsuite.Tagger()
tagger.open(path_export + '/ckip_tagger_model.crfsuite')

Let's tag a sentence to see how it works:

In [242]:
y_test_pred = []

for i in range(len(y_test)):
    new_pred = tagger.tag(X_test[i])
    y_test_pred.append(new_pred)

In [243]:
y_test_pred = cbyz.li_flatten(y_test_pred)
y_test = cbyz.li_flatten(y_test)

In [244]:
print(classification_report(y_pred=y_pred, y_true=y_test))
# print(classification_report(y_pred=y_pred, y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       165
           1       0.83      0.71      0.77         7

    accuracy                           0.98       172
   macro avg       0.91      0.85      0.88       172
weighted avg       0.98      0.98      0.98       172



## Let's check what classifier learned

In [ ]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

We can see that, for example, it is very likely that the beginning of an organization name (B-ORG) will be followed by a token inside organization name (I-ORG), but transitions to I-ORG from tokens with other labels are penalized. Also note I-PER -> B-LOC transition: a positive weight means that model thinks that a person name is often followed by a location.

Check the state features:

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

## Word2Vec

In [ ]:
import gensim
from gensim import matutils
from gensim.models.word2vec import Word2Vec

model = gensim.models.KeyedVectors.load_word2vec_format(
        path_resource + '/tmunlp_1.6B_WB_50dim_2020v1.bin.gz', 
        unicode_errors='ignore',
        binary=True
    )

In [ ]:
# Method 1

# wv_data = word_sentence_list
wv_data = word_sentence_list[0]

result = pd.DataFrame()

for i in range(len(wv_data)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

    # Create DF
    keys = list(word_vec.keys())
    vec_df = pd.DataFrame({'word':keys,
                          'index':range(len(keys))})

    vec_df = cbyz.df_cross_join(vec_df, vec_df)
    vec_df = vec_df[vec_df['index_x']<vec_df['index_y']] \
            .reset_index(drop=True) 
    
    for j in range(len(vec_df)):
        vec1 = model.get_vector(vec_df.loc[j, 'word_x'])
        vec2 = model.get_vector(vec_df.loc[j, 'word_y'])
        similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
        vec_df.loc[j, 'similarity'] = similarity

    result = result.append(vec_df)
    
    if i % 500 == 0:
        print(i, '/', len(word_sentence_list))
    
result = result.reset_index(drop=True)

In [ ]:
# Method 2

wv_data = word_sentence_list
# wv_data = word_sentence_list[0]

result = pd.DataFrame()

for i in range(len(wv_data)):

    word_vec = {}
    for w in word_sentence_list[i]:
        try:
            word_vec[w] = model.get_vector(w)
        except:
            pass

    # Create DF
    keys = list(word_vec.keys())
    vec_df = pd.DataFrame({'word':keys,
                          'index':range(len(keys))})

    for j in range(1, len(vec_df)):
        vec1 = model.get_vector(vec_df.loc[j, 'word'])
        vec2 = model.get_vector(vec_df.loc[j-1, 'word'])
        similarity = np.dot(matutils.unitvec(vec1), matutils.unitvec(vec2))
        vec_df.loc[j, 'similarity'] = similarity

    result = result.append(vec_df)
    
    if i % 500 == 0:
        print(i, '/', len(word_sentence_list))
    
result = result.reset_index(drop=True)

In [ ]:
word_sentence_list[0]

In [ ]:
result.to_excel(path_export + '/wv_result.xlsx', index=False, encoding='utf-8-sig')

In [ ]:
寶格麗
馨香
浪漫
玫香
淡香精

bvlgari
寶格麗
晶澈
女性
淡香水


In [ ]:
result

In [ ]:
# https://en.wikipedia.org/wiki/Unicode_block
# Emoticons 

# import re
# regex = '\u3000-\u303F|\u2000-\u206F'
# regex = '\u2000-\u206F'
# regex = '\u4E00-\u9FA5|\u2000-\u206F'

# string = 'A臺中市->'
# # filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
# filtrate = re.compile(u'[^(' + regex + ')]') # non-Chinese unicode range
# string = filtrate.sub(r'', string) # remove all non-Chinese characters

# string

In [ ]:
result[result['similarity']>0.5]

In [ ]:
type(model)

In [ ]:
# model.vectors
# model.vocab

In [ ]:
len(model.vocab)

## Worklist

In [ ]:
# - 跟淡香水之間的距離
#   和男性、女性、中性的距離
# - 如果title中同時出現「淡香水」、「淡香精」就排除